In [1]:
#TODO add STRUCT variables from dwarf info
#TODO MIN_ADDRESS, MAX_ADDRESS concept is not going to work always.(for data sections) 

from elftools.elf.elffile import ELFFile
from elftools.dwarf.descriptions import (
    describe_DWARF_expr, set_global_machine_arch)
from elftools.dwarf.locationlists import (
    LocationEntry, LocationExpr, LocationParser)
import posixpath
import sys,os,pickle
from elftools.elf.segments import Segment
from elftools.dwarf.locationlists import LocationParser, LocationExpr

from collections import defaultdict

import collections
import posixpath

import networkx as nx
from pyvis.network import Network
net = Network(notebook=True)
import matplotlib
import matplotlib.pyplot
from difflib import SequenceMatcher

import ntpath
from capstone import *
from capstone.x86 import *
import collections

import clang.cindex
from clang.cindex import CursorKind
import traceback, sys, magic, hashlib


sys.path.insert(0,'./align_utils')
from process_typedata_for_model import *
from dwarf_utils import *
from find_types import *
from clang_persers import *

from elf_utils import *
from dependency_utils import *
from illustrate_utils import *
# from var_and_instruction_realtions_utls import *
from poof import *

ANALYSIS_DATA_PATH = '/home/nahid/dataset/analysis_pkl_files/files/'
SRC_N_BIN_PATH = '/home/nahid/dataset/clones/'
ILLUSTRATION_LOG_PATH = "/home/nahid/dataset/illustration2/"
TYPE_DATA_SAVE_PATH = '/home/nahid/dataset/instructions_and_type_data/'



/home/nahid/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ELF_FILE_PATHS = find_elf_files(SRC_N_BIN_PATH)

# with open('ELF_FILE_PATHS.pkl', 'wb') as file:
#     pickle.dump(ELF_FILE_PATHS, file)

# Open the file in binary mode
with open('ELF_FILE_PATHS.pkl', 'rb') as file:
    # Call load method to deserialze
    ELF_FILE_PATHS = pickle.load(file)
    
print(len(ELF_FILE_PATHS))

57034


In [3]:



error_log = open("error.log", "w")
counter = 0
for binary_path in (ELF_FILE_PATHS) : 
    
#     if binary_path not in ['/home/nahid/dataset/clones/jeetendrabhattad_____C_Programs/file_handling/file_read_elf_file_',
#                            '/home/nahid/dataset/clones/jeetendrabhattad_____C_Programs/union/test_union_elf_file_',
#                           '/home/nahid/dataset/clones/jeetendrabhattad_____C_Programs/programs/string_extra_space_remove_elf_file_']:
#         continue
        
        
    unique_path = binary_path.split('clones')[1]
    unique_pkl_file_name=(hashlib.md5(unique_path.encode())).hexdigest()
    analysed_pkl_path = os.path.join( ANALYSIS_DATA_PATH ,unique_pkl_file_name+'.pkl')
    
    if os.path.isfile(analysed_pkl_path) == False:#no analysis file present
        continue

    binFileName = os.path.basename(binary_path)
    

    
    if check_dwarf_ok(binary_path)== False:
        continue
    
    print(" *_* "*10)
    print('Processing file:', binary_path)
    
    MIN_ADDRESS, MAX_ADDRESS =  get_min_max_address(binary_path)


    #Load the analysed data
    with (open(analysed_pkl_path , "rb")) as openfile:
        bb_data , ins_data , tool_addresses_list = pickle.load(openfile)
    
    try:
        VALID_INSTRUCTIONS_SET = get_valid_instructions(binary_path,tool_addresses_list,min_address=MIN_ADDRESS, max_address=MAX_ADDRESS)
        connected_addrs_and_program_slice = process_graphs(ins_data,MIN_ADDRESS,MAX_ADDRESS,VALID_INSTRUCTIONS_SET,bb_data)



    ##########################################################
    ################ CREATE HELPER DATA STRUCTURES   #########
    ##########################################################
    

        # create matrix for finding lineinfo for each address
        lineinfo_address_subprogram_complete = produce_address_to_lineinfo_matrix(binary_path , MIN_ADDRESS, MAX_ADDRESS)

#         FUNC_PARAMS = parse_dwarf_to_get_func_params('/home/nahid/reverse/binaries/c_many/stacktest' ,SRC_N_BIN_PATH)
#         print(FUNC_PARAMS)
#         break
        
    
    
    
        FUNC_PARAMS = parse_dwarf_to_get_func_params(binary_path , SRC_N_BIN_PATH)


#         if len(list(FUNC_PARAMS['structs'].keys()))==0:
#             continue
        
        variables_in_line = create_variable_per_line_matrix(binary_path, FUNC_PARAMS)
#         print(variables_in_line)

        line_to_address_matrix = build_line_to_relatedAddresses_matrix(lineinfo_address_subprogram_complete,VALID_INSTRUCTIONS_SET)
#         print('\n\n',line_to_address_matrix)
        ##############################################################################
        ##########################################################################
        inst_type_info = do_magic(VALID_INSTRUCTIONS_SET, FUNC_PARAMS,line_to_address_matrix ,variables_in_line)
        print('\n\n',inst_type_info,'\n\n\n\n')
        write_illustrated_file(binFileName ,lineinfo_address_subprogram_complete , inst_type_info,VALID_INSTRUCTIONS_SET,ILLUSTRATION_LOG_PATH)
        
        ###################################################################################
        process_data_4_model_and_save(VALID_INSTRUCTIONS_SET , connected_addrs_and_program_slice,inst_type_info,unique_pkl_file_name)
        
    
    except Exception as e:#TODO, solve the most frequent errors
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        print(traceback.format_exc(),'\n\n\n\n')
        error_log.write('\n\n')
        error_log.write(' analysed_pkl_path: '+ analysed_pkl_path + '\n')
        error_log.write(str(exc_type) +" fname: "+fname + " lineno: "+ str(exc_tb.tb_lineno) )
    counter+=1
    
    

 *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_* 
Processing file: /home/nahid/dataset/clones/Princexz_____alx-low_level_programming/0x04-more_functions_nested_loops/9-fizz_buzz_elf_file_
Low PC:  0x1169  High PC 0x120
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:17  var:s{'i': -20}  ins:{'0x1175': -4}
line:17  var:s{'i': -20}  ins:{}
line:19  var:s{'i': -20}  ins:{'0x1181': -4}
line:19  var:s{'i': -20}  ins:{}
line:20  var:s{}  ins:{}
line:21  var:s{'i': -20}  ins:{'0x11c6': -4}
line:21  var:s{'i': -20}  ins:{}
line:22  var:s{}  ins:{}
line:23  var:s{'i': -20}  ins:{'0x1202': -4}
line:23  var:s{'i': -20}  ins:{}
line:24  var:s{}  ins:{}
line:26  var:s{'i': -20}  ins:{'0x1241': -4}
line:27  var:s{'i': -20}  ins:{'0x125a': -4}
line:28  var:s{}  ins:{}
line:17  var:s{'i': -20}  ins:{'0x126a': -4}
line:17  var:s{'i': -20}  ins:{'0x126e': -4}
line:30  var:s{}  ins:{}
DBG:  calculated offse

x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:14  var:s{}  ins:{}
line:14  var:s{}  ins:{}
line:15  var:s{'n': -20}  ins:{}
line:15  var:s{'n': -20}  ins:{'0x11d0': -4}
line:16  var:s{'n': -20}  ins:{'0x11d3': -4}
line:17  var:s{'n': -20}  ins:{'0x11d9': -4}
line:18  var:s{'n': -20}  ins:{'0x11f4': -4}
line:19  var:s{'n': -20}  ins:{'0x11fa': -4}
line:21  var:s{'n': -20}  ins:{'0x1215': -4}
DBG:  calculated offset:  -16
DBG:  line:  14    inst_matrix:  {}    var_matrix   {}
DBG:  line:  14    inst_matrix:  {}    var_matrix   {}
DBG:  line:  15    inst_matrix:  {}    var_matrix   {'n': -20}
DBG:  line:  15    inst_matrix:  {'0x11d0': -4}    var_matrix   {'n': -20}
DBG:  line:  16    inst_matrix:  {'0x11d3': -4}    var_matrix   {'n': -20}
DBG:  line:  17    inst_matrix:  {'0x11de': 3615, '0x11d9': -4}    var_matrix   {'n': -20}
DBG:  line:  18    inst_matrix:  {'0x11f4': -4}    var_matrix 

Low PC:  0x1149  High PC 0x81
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:11  var:s{'num': -20}  ins:{'0x1155': -4}
line:11  var:s{'num': -20}  ins:{}
line:13  var:s{'num': -20}  ins:{'0x115e': -4}
line:13  var:s{'num': -20}  ins:{'0x1188': 48}
line:14  var:s{'num': -20}  ins:{'0x1192': -4}
line:16  var:s{}  ins:{}
line:17  var:s{}  ins:{}
line:11  var:s{'num': -20}  ins:{'0x11af': -4}
line:11  var:s{'num': -20}  ins:{'0x11b3': -4}
line:20  var:s{}  ins:{}
DBG:  calculated offset:  -16
DBG:  line:  11    inst_matrix:  {'0x1155': -4}    var_matrix   {'num': -20}
DBG:  line:  11    inst_matrix:  {}    var_matrix   {'num': -20}
DBG:  line:  13    inst_matrix:  {'0x115e': -4}    var_matrix   {'num': -20}
DBG:  line:  13    inst_matrix:  {'0x1188': 48}    var_matrix   {'num': -20}
DBG:  line:  14    inst_matrix:  {'0x1192': -4}    var_matrix   {'num': -20}
DBG:  line:  16    inst_matrix:  {

Low PC:  0x1149  High PC 0x13d
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:11  var:s{'i': -24}  ins:{'0x1155': -8}
line:11  var:s{'i': -24}  ins:{}
line:13  var:s{'j': -20}  ins:{'0x1161': -4}
line:13  var:s{'j': -20}  ins:{}
line:15  var:s{'j': -20, 'i': -24}  ins:{'0x1170': -4, '0x116d': -8}
line:17  var:s{'i': -24}  ins:{'0x1179': -8}
line:17  var:s{'i': -24}  ins:{}
line:18  var:s{'i': -24}  ins:{'0x11a0': -8}
line:18  var:s{'i': -24}  ins:{'0x11ca': 48}
line:19  var:s{}  ins:{}
line:20  var:s{'j': -20}  ins:{'0x11de': -4}
line:20  var:s{'j': -20}  ins:{}
line:21  var:s{'j': -20}  ins:{'0x1205': -4}
line:21  var:s{'j': -20}  ins:{'0x122f': 48}
line:22  var:s{'j': -20, 'i': -24}  ins:{'0x1239': -8}
line:22  var:s{'j': -20, 'i': -24}  ins:{'0x123f': -4}
line:24  var:s{}  ins:{}
line:25  var:s{}  ins:{}
line:13  var:s{'j': -20}  ins:{'0x1259': -4}
line:13  var:s{'j': -20}  ins:{'0x125

Low PC:  0x1149  High PC 0x69
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:10  var:s{}  ins:{}
line:11  var:s{}  ins:{}
line:12  var:s{}  ins:{}
line:13  var:s{}  ins:{}
line:14  var:s{}  ins:{}
line:15  var:s{}  ins:{}
line:16  var:s{}  ins:{}
line:17  var:s{}  ins:{}
line:18  var:s{}  ins:{}
DBG:  calculated offset:  None


 {} 




 *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_* 
Processing file: /home/nahid/dataset/clones/Princexz_____alx-low_level_programming/0x02-functions_nested_loops/102-fibonacci_elf_file_
Low PC:  0x1169  High PC 0x96
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:12  var:s{'sum': -24, 'fib2': -32, 'fib1': -40}  ins:{'0x1175': -24}
line:12  var:s{'sum': -24, 'fib2': -32, 'fib1': -40}  ins:{'0x117d': -16}
line:14  var:s{'count': -44}  ins:{'0x1185': -28}
line:14

...

Low PC:  0x1162  High PC 0x1a
Low PC:  0x1149  High PC 0x19
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:7  var:s{}  ins:{}
DBG:  calculated offset:  None
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:12  var:s{}  ins:{}
DBG:  calculated offset:  None


 {} 




 *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_* 
Processing file: /home/nahid/dataset/clones/jeetendrabhattad_____C_Programs/storage_classes-2/auto_storage_class_test_elf_file_
Low PC:  0x11a7  High PC 0x23
Low PC:  0x1149  High PC 0x5e
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:6  var:s{'a': -20}  ins:{'0x1155': -4}
line:8  var:s{}  ins:{}
line:9  var:s{'a': -20}  ins:{'0x116b': -4}
line:10  var:s{}  ins:{}
line:11  var:s{'a':

 *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_* 
Processing file: /home/nahid/dataset/clones/jeetendrabhattad_____C_Programs/pointers-2/little_endian_elf_file_
Low PC:  0x1169  High PC 0x19c
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:6  var:s{'i': -36}  ins:{'0x1184': -20}
line:7  var:s{'ptr': -32, 'i': -36}  ins:{'0x118f': -16, '0x118b': -20}
line:9  var:s{'i': -36}  ins:{'0x1193': -20}
line:10  var:s{'ptr': -32}  ins:{'0x11ae': -16}
line:10  var:s{'ptr': -32}  ins:{'0x11b8': -16}
line:11  var:s{'ptr': -32}  ins:{'0x11d3': -16}
line:11  var:s{'ptr': -32}  ins:{'0x11dd': -16}
line:12  var:s{'ptr': -32}  ins:{'0x11f8': -16}
line:13  var:s{'ptr': -32}  ins:{'0x11fd': -16}
line:13  var:s{'ptr': -32}  ins:{'0x1207': -16}
line:14  var:s{'ptr': -32}  ins:{'0x1222': -16}
line:14  var:s{'ptr': -32}  ins:{'0x122c': -16}
line:15  var:s{'ptr': -32}  ins:{'0x1247': -16}
line:16  var:s{'ptr': -3

Low PC:  0x14f1  High PC 0x23
Low PC:  0x14be  High PC 0x33
Low PC:  0x13a2  High PC 0x11c
Low PC:  0x12cc  High PC 0xd6
Low PC:  0x1240  High PC 0x8c
Low PC:  0x1189  High PC 0xb7
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:22  var:s{'ptr': -24}  ins:{'0x119c': -8}
line:23  var:s{}  ins:{}
line:24  var:s{'ptr': -24}  ins:{'0x11be': -8}
line:30  var:s{'ptr': -24}  ins:{'0x11d9': -8}
line:31  var:s{'ptr': -24}  ins:{'0x11de': -8}
line:33  var:s{'ptr': -24}  ins:{'0x1200': -8}
line:34  var:s{'ptr': -24}  ins:{'0x1204': -8}
line:35  var:s{}  ins:{}
DBG:  calculated offset:  -16
DBG:  line:  22    inst_matrix:  {'0x1195': 3716, '0x119c': -8}    var_matrix   {'ptr': -24}
DBG:  line:  23    inst_matrix:  {'0x11a0': 3929, '0x11aa': 3701}    var_matrix   {}
DBG:  line:  24    inst_matrix:  {'0x11c5': 3685, '0x11be': -8}    var_matrix   {'ptr': -24}
DBG:  line:  30    inst_matrix:  {'0x11d9': -

x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:31  var:s{'k': -48}  ins:{'0x119c': -6, '0x1195': -10, '0x118d': -18, '0x1186': -22, '0x1182': -23, '0x117d': -31, '0x1175': -32}
line:38  var:s{}  ins:{}
line:39  var:s{}  ins:{}
line:40  var:s{}  ins:{}
DBG:  calculated offset:  None


 {} 




 *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_* 
Processing file: /home/nahid/dataset/clones/jeetendrabhattad_____C_Programs/structure/bit_fields_test_elf_file_
Low PC:  0x1149  High PC 0x82
x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   x  x   
line:9  var:s{}  ins:{'0x1155': -4}
line:10  var:s{}  ins:{'0x117c': -4}
line:10  var:s{}  ins:{}
line:10  var:s{}  ins:{'0x118a': -4}
line:10  var:s{}  ins:{'0x1197': -4}
DBG:  calculated offset:  None


 {} 




 *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_*  *_* 
Processi

KeyboardInterrupt: 

In [ ]:
get_type_name

In [ ]:
print("SUBHAN'ALLAH")

In [ ]:
a= {'dog': {'name': {'type': '*char', 'location': 0, 'offset': 0}}}

In [ ]:
len(list(a.keys()))